# 🚀 İnteraktif JetX Model Eğitmeni (v4.0)
Bu not defteri, JetX tahmin sistemi için **interaktif** ve tam otomatik bir eğitim akışı sağlar.

## ✨ Yeni Özellikler (v4.0)
- **İnteraktif Model Seçimi:** Kod yazmadan, sadece onay kutularını (checkbox) işaretleyerek eğitmek istediğiniz modelleri seçin.
- **Otomatik HPO & MLflow:** `Optuna` ve `MLflow` entegrasyonları ile en iyi hiperparametreler bulunur ve tüm süreç izlenir.
- **Kararlı Çalışma:** Önceki sürümlerdeki kritik hatalar giderildi ve yeni doğruluk metriği eklendi.

**⚠️ Başlamadan Önce: Runtime > Change runtime type > GPU seçeneğini etkinleştirin!**

## 🛠️ 1. Sistem Kurulumu ve Projeyi Yükleme
Bu hücre, projeyi GitHub'dan klonlar (veya günceller) ve gerekli tüm kütüphaneleri yükler.

In [ ]:
import os
import sys
import subprocess

def setup_environment():
    """Clones or updates the repository and sets up the environment."""
    repo_url = "https://github.com/onndd/predictor_1.git"
    project_dir = "/content/predictor_1"

    if os.path.exists(project_dir):
        print("📁 Proje zaten mevcut. Güncelleniyor...")
        os.chdir(project_dir)
        subprocess.run(["git", "pull", "origin", "main"], check=True, capture_output=True)
    else:
        print("📥 GitHub'dan proje klonlanıyor...")
        os.chdir("/content")
        subprocess.run(["git", "clone", repo_url], check=True, capture_output=True)
        os.chdir(project_dir)

    # Add src to Python path
    src_path = os.path.join(project_dir, "src")
    if src_path not in sys.path:
        sys.path.insert(0, src_path)
        print(f"✅ {src_path} Python path'e eklendi")

    # Install dependencies
    requirements_path = os.path.join(project_dir, 'requirements_enhanced.txt')
    if os.path.exists(requirements_path):
        print("📦 Gerekli kütüphaneler yükleniyor...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-r", requirements_path])
        print("✅ Kütüphaneler başarıyla yüklendi.")

setup_environment()
print("🎉 Kurulum tamamlandı!")

## 📊 2. MLflow Deney Takibini Başlatma
Bu hücre, MLflow arayüzünü başlatır ve `ngrok` kullanarak size erişebileceğiniz bir public URL verir. Lütfen aşağıdaki hücreye ngrok authtoken'ınızı girin.

In [ ]:
from getpass import getpass
from colab.health_check import ColabHealthCheck
import time

# ngrok authtoken'ınızı buraya girin
NGROK_AUTH_TOKEN = getpass('Lütfen ngrok authtokeninizi girin: ')

# ngrok bağlantısını önceden test et
health_checker = ColabHealthCheck()
ngrok_ok, error_message = health_checker.check_ngrok_tunnel(NGROK_AUTH_TOKEN)

if ngrok_ok:
    print("✅ ngrok bağlantı testi başarılı.")
    
    # MLflow UI'ı arka planda başlat
    get_ipython().system_raw('mlflow ui --host 0.0.0.0 &')
    
    # Give MLflow a moment to start
    time.sleep(5)
    
    print("\n✅ MLflow arayüzü için public URL oluşturuluyor...")
    print("👇 Aşağıdaki linke tıklayarak deneyleri izleyebilirsiniz:")
    
    try:
        from pyngrok import ngrok
        public_url = ngrok.connect(5000, "http").public_url
        print(f"--> {public_url}")
    except Exception as e:
        print(f"❌ ngrok tüneli oluşturulamadı. Hata: {e}")
else:
    print(f"\n❌ ngrok ön kontrolü başarısız oldu. Lütfen authtoken'ınızı ve internet bağlantınızı kontrol edin.")
    print(f"   Hata Detayı: {error_message}")

## 🚀 3. Modelleri Seç ve Eğitimi Başlat
Aşağıdaki arayüzü kullanarak eğitmek istediğiniz modelleri seçin ve butona tıklayarak eğitimi başlatın.

In [ ]:
import torch
import ipywidgets as widgets
from IPython.display import display
from src.training.master_trainer import MasterTrainer
from src.config.settings import get_aggressive_training_profiles
import traceback

# Eğitilebilir modelleri al
available_models = list(get_aggressive_training_profiles().keys())

# Arayüz elemanlarını oluştur
model_checkboxes = [widgets.Checkbox(value=False, description=model) for model in available_models]
start_button = widgets.Button(description="Eğitimi Başlat", button_style='success', icon='rocket')
output_area = widgets.Output()

def on_button_clicked(b):
    with output_area:
        output_area.clear_output()
        selected_models = [cb.description for cb in model_checkboxes if cb.value]
        
        if not selected_models:
            print("⚠️ Lütfen en az bir model seçin!")
            return
        
        print(f"🚀 Seçilen modellerle eğitim başlatılıyor: {', '.join(selected_models)}")
        
        # Cihaz tespiti
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        if device == 'cuda':
            print(f"✅ GPU bulundu: {torch.cuda.get_device_name(0)}")
        else:
            print("⚠️ GPU bulunamadı. Eğitim CPU ile devam edecek.")
            
        try:
            master_trainer = MasterTrainer(models_to_train=selected_models, device=device)
            master_trainer.run()
        except Exception as e:
            print(f"❌ Eğitim sırasında beklenmedik bir hata oluştu: {e}")
            traceback.print_exc()

start_button.on_click(on_button_clicked)

# Arayüzü göster
print("Eğitmek istediğiniz modelleri seçin:")
ui = widgets.VBox(model_checkboxes + [start_button, output_area])
display(ui)